In [2]:
import numpy as np
import pandas as pd


In [3]:
def calcExpected(player1rank,player2rank,totalScore):
    #As ranking increase you expect to win a greater proportion of points
    #Checks the proportion of total points to give you your expected point total.
    #This means similar rankings expect close games thus little change
    #Really different rankigns expect a much greater score difference. 
    #Example: 600 beats 400; 21-10
    #If you want to make the ranks more sparse, you can reduce starting values
    #Or increase the variation
    #There is a function below so you can see the effect of adjusting the variation
    variation=400
    expected=totalScore/(1+10**((player1rank-player2rank)/variation))
    return(expected)

#REDUNDENT with new method. Use Predicted score difference instead

In [147]:
#Only to verify above procedure. 
#If these scores were reached there shouldn't be any change to peoples ranks.
def predictScore(rank1,rank2):
    variation=670
    expected=1/(1+10**((rank1-rank2)/variation))
    if expected<0.475:
        score=int((21*expected)/(1-expected))
        return(21,score)
    elif expected<0.5:
        score=int(min((2*expected)/(1-2*expected),29))
        return(min(score+2,30),score)
    elif expected==0.5:
        return(21,21)
    elif expected<0.525:
        expected=1-expected
        score=int(min((2*expected)/(1-2*expected),29))
        return(score,min(score+2,30))
    elif expected<1:
        expected=1-expected
        score=int((21*expected)/(1-expected))
        return(score,21)
    else:
        print("Not possible to compute")
        return(0,0)

In [148]:
def calcExpectedPointDiff(player1rank,player2rank):
    x=predictScore(player1rank,player2rank)
    return(x[0]-x[1])

In [206]:
def calcRanks():
    dic={}
    # This is to import the data
    dataset=pd.read_csv('testdata.csv')
    
    #Sensitivity adjusts how drastically the scores change per game
    sen=7
    
    #Iterates over each game. Must be chronological
    for i,(col1,score1,score2,col2) in dataset.iterrows():
        #This formats the names so that it doesnt matter if you added a spacebar
        name1=col1.strip().upper()
        name2=col2.strip().upper()
        
        #This creates a new player in the dictionary if it hasn't been seen before.
        if name1 not in dic:
            dic[name1]=400
        if name2 not in dic:
            dic[name2]=400
            
        #Calculates expected score as a function of current rankings, and total score of the game
        #escore1=calcExpected(dic[name1],dic[name2],score1+score2)
        #escore2=calcExpected(dic[name2],dic[name1],score1+score2)
        
        #The change is then added and subtracted accordingly
        #change1=int(sen*(score1-escore1))
        #change2=int(sen*(score2-escore2))
        #dic[name1]=1+dic[name1]+change1
        #dic[name2]=1+dic[name2]+change2
        
        #Used score to represent points won minus points lost
        pointDiff=score1-score2
        ePointDiff=calcExpectedPointDiff(dic[name1],dic[name2])
        change=sen*(pointDiff-ePointDiff)
        dic[name1]=2+dic[name1]+change
        dic[name2]=2+dic[name2]-change
    return(dic)

In [207]:
def orderPlayers(elo):
    players=[str(A) for A in elo]
    for index in range(1,len(players)):
    #index is the one we are inserting (using insertion sort)
        player=players[index]
        value=elo[player]
        i=index-1
        while i>=0:
            #if score is more than the person to the left, swap, keep doing till done
            if value>elo[players[i]]:
                players[i+1]=players[i]
                players[i]=player
                i=i-1
            else:
                break
    scores=[elo[plyr] for plyr in players]
    return(players,scores)

In [208]:
eloScores=calcRanks()
orderPlayers(eloScores)


(['ANUJ', 'C', 'B', 'F', 'H', 'D', 'E', 'G'],
 [581, 553, 471, 469, 438, 346, 280, 226])

In [199]:
#You can check the prediced score using player ranks as follows.
#Adjust this my changing variation in predict scores
#higher variation predicts closer game => ranks will be all closer together
#If we adjust variation such that a 200 difference in rank, gives us 21-5. Use 340. For 21-10 use 670
predictScore(1000,237)

(21, 1)